In [1]:
import redis
import json

# Connect to Redis
r = redis.Redis(
    host='redis',  # or your Redis server's hostname
    port=6379,         # default Redis port
)

In [2]:
r.keys('*')

[b'startup_lock',
 b'saq:default:schedule',
 b'saq:default:sweep',
 b'saq:default:stats:10a92a7c-cb48-11ee-9d2c-0242ac1d0004',
 b'saq:default:stats:10ac59ae-cb48-11ee-b9cf-0242ac1d0004',
 b'saq:default:stats:10a91b86-cb48-11ee-ada5-0242ac1d0004',
 b'saq:default:stats',
 b'saq:default:incomplete',
 b'saq:job:default:cron:handle_finished_calcs']

In [3]:
for key in r.scan_iter("saq:job:default:*"):
    print(key)

b'saq:job:default:cron:handle_finished_calcs'


In [4]:
stats_data = {}

for key in r.scan_iter("saq:job:default:*"):
    # Determine the type of data stored at the key
    key_type = r.type(key).decode('utf-8')
    value = r.get(key)
    
    # Decode the key (if necessary) and store the retrieved data
    decoded_key = key.decode('utf-8')
    stats_data[decoded_key] = value

    for key, value in stats_data.items():
        data_str = value.decode('utf-8')
        data_dict = json.loads(data_str)

        if (data_dict['status'] == 'complete'):
            print(f"Job {key} completed")
        elif (data_dict['status'] == 'failed'):
            print(f"Job {key} failed")
        elif (data_dict['status'] == 'active'):
            print(f"Job {key} active")

In [5]:
data_bytes = stats_data['saq:default:stats:10a92a7c-cb48-11ee-9d2c-0242ac1d0004']
data_str = data_bytes.decode('utf-8')
data_dict = json.loads(data_str)

KeyError: 'saq:default:stats:409d3a14-c672-11ee-a06a-0242ac1d0004'

In [18]:
data_dict

{'complete': 0, 'failed': 0, 'retried': 0, 'aborted': 0, 'uptime': 443679512}

In [5]:
next_task = r.lindex('my_queue', 0)

In [6]:
next_task